In [107]:
from scipy.io import arff
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics as stat
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from collections import defaultdict
from datetime import  datetime

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15,5)

In [29]:
atom=pd.read_csv('new_atom.CSV')
bitcoin=pd.read_csv('new_bitcoin.CSV')
brew=pd.read_csv('new_brew.CSV')
ember=pd.read_csv('new_ember.CSV')
s_learn=pd.read_csv('new_scikit-learn.CSV')

first, define a function to exchange str to datatime of 'time' columns 

In [22]:
 def get_time(DDD):
    for data in DDD:
        for i in range(len(data['time'])):
            error=False
            a=list()
            try:
                data['time'][i]=datetime.strptime(data['time'][i],'%Y-%m-%dT%H:%M:%SZ')
            except:
                a.append([i,data['time'][i]])
                error=True
    return a

In [31]:
sour_data=[atom,bitcoin,brew,ember,s_learn]
get_time(sour_data)

[[30498, nan]]

sort by time

In [32]:
for i in sour_data:
    i=i.sort_values(by=['time'])

In [33]:
atom.sort_values(by=['time'])

,comment_ID,issue_no,commenter_ID,commenter,is_author,commenter_type,commenter_role,time,sentiment_list,sentiment,pred_label_x,comment_content,word_count,has_link,pred_label_y
0,7800113.0,5,173,bkeepers,NaN,User,CONTRIBUTOR,2012-08-16 21:27:27,[1],1.000000,NaN,Fixed in 60eb4a8a,5.0,NaN,N
1,8634446.0,39,1789,nathansobo,NaN,User,CONTRIBUTOR,2012-09-17 23:15:42,"[1, 0, 1]",0.666667,NaN,This is probably the old CEF 1 build of Atom.....,51.0,NaN,N
2,8664521.0,39,1789,nathansobo,NaN,User,CONTRIBUTOR,2012-09-18 18:17:34,[2],2.000000,NaN,Thank you!,3.0,NaN,N
3,8843458.0,39,1789,nathansobo,NaN,User,CONTRIBUTOR,2012-09-25 04:41:37,"[0, 1, 2]",1.000000,NaN,No it definitely shouldn't! I'll look into it ...,19.0,NaN,N
4,8864412.0,39,1789,nathansobo,NaN,User,CONTRIBUTOR,2012-09-25 18:16:04,"[1, 0, 2, 1]",1.000000,NaN,@cobyism I'm having some trouble reproducing y...,26.0,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52126,2874.0,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52127,2855.0,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52128,1433.0,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52129,3799.0,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
issue_title=['issue_no','comment_ID_list','LDi_list','comment_time','commer_per_list','time_gap','time_gap_list','LD1','LD2','LD3','LD4','LD5','LD6','N','total']
test_data=pd.DataFrame(columns=issue_title)
test_data.head()

,issue_no,comment_ID_list,LDi_list,comment_time,commer_per_list,time_gap,time_gap_list,LD1,LD2,LD3,LD4,LD5,LD6,N,total


In [99]:
from collections import Counter

In [102]:
a=['a','a','a','a','a','a','a','a','a','b','b','b','b','b',]
A=Counter(a)
A['a']

9

In [105]:
def get_issue(datas):
    datas['issue_no'].value_counts().to_dict()
    issue_data=pd.DataFrame(columns=issue_title)
    counttt=0
    for data in datas.iterrows():
        counttt+=1
        print(counttt)
        issue_no=data[1]['issue_no']
        comment_ID_list=datas[datas['issue_no']==data[1]['issue_no']].sort_values(by=['time']).comment_ID.to_list()
        LDi_list=datas[datas['issue_no']==data[1]['issue_no']].sort_values(by=['time']).pred_label_y.to_list()
        comment_time=datas[datas['issue_no']==data[1]['issue_no']].sort_values(by=['time']).time.to_list()
        commer_per_list=list()
        time_gap_list=list()
        time_gap=0
        if len(comment_time)>1:
            time_gap=comment_time[-1]-comment_time[0]
            for i in range(1,len(comment_time)):
                commer_per_list.append([LDi_list[i-1],LDi_list[i]])
                time_gap_list.append((comment_time[i]-comment_time[i-1]).seconds)
        num=Counter(LDi_list)
        total=len(comment_time)
        try1={'issue_no':issue_no,
        'comment_ID_list':comment_ID_list,
        'LDi_list':LDi_list,
        'comment_time':comment_time,
        'commer_per_list':commer_per_list,
        'time_gap':time_gap,
        'time_gap_list':time_gap_list,
        'LD1':num['LD1'],
        'LD2':num['LD2'],
        'LD3':num['LD3'],
        'LD4':num['LD4'],
        'LD5':num['LD5'],
        'LD6':num['LD6'],
        'N':num['N'],
        'total':total}
        issue_data.append(try1,ignore_index=True)
    return issue_data

In [96]:
temp=pd.DataFrame(data={'issue_no':'issue_no',
        'comment_ID_list':'comment_ID_list',
        'LDi_list':'LDi_list',
        'comment_time':'comment_time',
        'commer_per_list':'commer_per_list',
        'time_gap':'time_gap',
        'time_gap_list':'time_gap_list',
        'LD1':'LD1',
        'LD2':'LD2',
        'LD3':'LD3',
        'LD4':'LD4',
        'LD5':'LD5',
        'LD6':'LD6',
        'N':'N',
        'total':'total'},columns=issue_title,index=[0])
temp.append(a,ignore_index=True)

,issue_no,comment_ID_list,LDi_list,comment_time,commer_per_list,time_gap,time_gap_list,LD1,LD2,LD3,LD4,LD5,LD6,N,total
0,issue_no,comment_ID_list,LDi_list,comment_time,commer_per_list,time_gap,time_gap_list,LD1,LD2,LD3,LD4,LD5,LD6,N,total
1,5,[7800113.0],[N],[2012-08-16 21:27:27],[],0,[],0,0,0,0,0,0,1,1


In [59]:
atom.issue_no.value_counts()

2956     257
2654     242
6867     186
17672    153
5344     148
        ... 
7100       1
11684      1
8018       1
6280       1
13650      1
Name: issue_no, Length: 13824, dtype: int64

In [113]:
import csv
from collections import defaultdict
from datetime import  datetime


In [114]:

def csv_dict_write(path,head,data):
    with open(path,'w',encoding='utf-8',newline='') as f:
        writer = csv.DictWriter(f,head)
        writer.writeheader()
        writer.writerows(data)
        return True


In [115]:

def csv_dict_read(path):
    f=open(path,'r',encoding='utf-8')
    reader = csv.DictReader(f,dialect='excel')
    return reader


In [134]:

def get_lable(path):
    reader = csv_dict_read(path)
    data=[]
    LDi_list=defaultdict(list)
    comment_ID_list=defaultdict(list)
    comment_time=defaultdict(list)
    comment_timedata=defaultdict(list)
    commer_per_list=defaultdict(list)
    time_gap=0
    time_gap_list=defaultdict(list)
    head=['issue_no','comment_ID_list','LDi_list','comment_time','commer_per_list','time_gap','time_gap_list','LD1','LD2','LD3','LD4','LD5','LD6','N','total']
    for row in reader:
        try:
            time=datetime.strptime(row['time'],'%Y-%m-%dT%H:%M:%SZ')
        except:
            pass
        LDi_list[row['issue_no']].append(row['pred_label_y'])
        comment_ID_list[row['issue_no']].append(row['comment_ID'])
        comment_time[row['issue_no']].append(row['time'])
        comment_timedata[row['issue_no']].append(time)
    if len(comment_ID_list)>1:
        try:
            time_gap=(comment_timedata[-1]-comment_timedata[0]).seconds
        except:
            time_gap='bad'
        for i in range(1,len(comment_ID_list)):
            commer_per_list[row['issue_no']].append([comment_ID_list[i-1],comment_ID_list[i]])
            try:
                time_gap_list[row['issue_no']]=(comment_timedata[i]-comment_timedata[i-1]).seconds
            except:
                time_gap_list[row['issue_no']]='bad'
    for i,j in LDi_list.items():
        data.append({'issue_no':i,
        'comment_ID_list':comment_ID_list[i],
        'LDi_list':LDi_list[i],
        'comment_time':comment_time[i],
        'commer_per_list':commer_per_list[i],
        'time_gap':time_gap,
        'time_gap_list':time_gap_list[i],
        'LD1':j.count('LD1'),
        'LD2':j.count('LD2'),
        'LD3':j.count('LD3'),
        'LD4':j.count('LD4'),
        'LD5':j.count('LD5'),
        'LD6':j.count('LD6'),
        'N':j.count('N'),
        'total':len(j)})
    csv_dict_write(path[:-4]+'_lab'+'.CSV',head,data)


In [126]:
def creat_lable():
    get_lable('new_atom.CSV')
    get_lable('new_bitcoin.CSV')
    get_lable('new_brew.CSV')
    get_lable('new_ember.CSV')
    get_lable('new_scikit-learn.CSV')

In [135]:
creat_lable()

KeyError: '\ufeffcomment_ID'